In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#%matplotlib widget

In [2]:
scada_data = pd.read_csv("C:\\Users\\hanna\\Desktop\\data\\Events.csv", delimiter=',',  parse_dates=True)


In [3]:
scada_data.drop(columns=['timestamp ($ts)_DateTime','payload_quality_String', 'payload_timestamp_DateTime', 'payload_unit_String', 'payload_quantity_String'], inplace=True)
scada_data.columns
scada_data.rename(columns={'timestamp  (UTC+00:00) Local - Europe/London: GMT_DateTime':'timestamp',
                          'apiKey_String': 'turbine', 'measurement_String': 'measurement', 'payload_value_Double' : 'value'}, inplace=True)
scada_data['timestamp'] = pd.to_datetime(scada_data['timestamp'])

In [4]:
wind_speed = scada_data[scada_data['measurement'] == 'Windspeed'][["timestamp","turbine","value"]]
wind_speed.rename(columns={"value": "wind_speed"}, inplace=True)
wind_speed.reset_index(inplace=True, drop=True)
wind_direction = scada_data[scada_data['measurement'] == 'WindDirection'][["timestamp","turbine","value"]]
wind_direction.rename(columns={"value": "wind_direction"}, inplace=True)
wind_direction.reset_index(inplace=True, drop=True)
wind_speed['wind_direction'] = wind_direction['wind_direction']
data = wind_speed.copy()
data

,timestamp,turbine,wind_speed,wind_direction
0,2020-12-26 11:55:38.012,nor_var_t01,7.0,170.300003
1,2020-12-26 11:55:38.012,nor_var_t03,7.4,160.300003
2,2020-12-26 11:55:38.012,nor_var_t05,5.7,166.300003
3,2020-12-26 11:55:38.012,nor_var_t06,8.2,170.500000
4,2020-12-26 11:55:38.013,nor_var_t07,9.6,171.600006
...,...,...,...,...
604368,2021-01-25 11:55:10.250,nor_var_t05,1.3,37.400002
604369,2021-01-25 11:55:10.250,nor_var_t06,2.2,33.700001
604370,2021-01-25 11:55:10.250,nor_var_t07,2.2,28.000000
604371,2021-01-25 11:55:10.250,nor_var_t08,1.4,34.299999


In [5]:
data_rs = data.groupby(['timestamp']).mean()
data_rs = data_rs.resample('10min').first()
data_rs

,wind_speed,wind_direction
timestamp,,
2020-12-26 11:50:00,7.075000,166.850002
2020-12-26 12:00:00,7.385714,171.242859
2020-12-26 12:10:00,6.971429,173.671428
2020-12-26 12:20:00,7.250000,190.049995
2020-12-26 12:30:00,8.050000,190.250000
...,...,...
2021-01-25 11:10:00,2.614286,107.200003
2021-01-25 11:20:00,2.100000,186.900003
2021-01-25 11:30:00,2.400000,196.200003


In [19]:
from statsmodels.tsa.arima_model import ARIMA
import datetime as dt

X = data_rs[['wind_speed']][0:2000]
size = int(len(X) * 0.04)
train, test = X[0:size], X[size:len(X)]
history = train
predictions = pd.DataFrame(index=test.index, columns=['prediction'])
times = []
for i, row in test.iterrows():
    start = dt.datetime.now()
    model = ARIMA(history, order=(1,1,0))
    model_fit = model.fit(disp=0)
    output = model_fit.forecast()
    yhat = output[0].flatten()[0]
    predictions.loc[i]['prediction'] = yhat
    history.loc[i] = row['wind_speed']
    end = dt.datetime.now()
    times.append(end-start)
    #print('predicted=%f, expected=%f' % (predictions.loc[i]['prediction'], row['WindSpeedMean']))

print('done')
print((sum(times, dt.timedelta()) / len(times)))

done
0:00:00.030886


In [35]:
plt.figure(figsize=(20,5))
plt.plot(history, color='blue', label='real')
plt.plot(predictions, color='red', label='prediction')
plt.legend(loc='best')
plt.title('Real vs Predictions')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Real vs Predictions')

In [32]:
combined_df = pd.concat([history,predictions], axis=1).dropna()

In [33]:
from sklearn.metrics import mean_squared_error
error = mean_squared_error(combined_df['wind_speed'], combined_df['prediction'])
print('Test MSE: %.3f' % error)

Test MSE: 2.418


In [34]:
import random
import numpy
from matplotlib import pyplot

plt.hist([combined_df['wind_speed'], combined_df['prediction']], bins=list(range(0,15)), label=['Real', 'Prediction'])
plt.legend(loc='upper right')
plt.show()